In [1]:
import pyodbc
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import win32com.client as win32


In [2]:
if (date.today()- pd.DateOffset(days=1)).weekday() == 6:
    today = (date.today()- pd.DateOffset(days=3)).strftime('%Y-%m-%d')
else:
    today = (date.today()- pd.DateOffset(days=1)).strftime('%Y-%m-%d')
print(today)

2024-05-01


In [3]:
today = '2024-04-30'

### get external data

In [4]:
# connection
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_tbl_ops_shortage_ext where ReportDate = '{today}'")
shortage_ext = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
shortage_ext['HP_PN'] = shortage_ext['HP_PN'].str.replace('\n', ',', regex=False)
shortage_ext['FV'] = shortage_ext['FV'].str.replace('\n', ',', regex=False)

# download PNFV from SQL
cursor.execute("SELECT * FROM OPS.GPS_tbl_ops_PN_FV")
PNFV = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
PNFV = PNFV.rename({'Descr':'FV'})

In [5]:
# group bu commodity and count the len of each group
shortage_ext = shortage_ext[shortage_ext['P1'] != 0]
ext_count = shortage_ext.groupby(['Commodity']).size().reset_index(name='Count')

# CPU need to add supplier
ext_CPU = shortage_ext[shortage_ext['Commodity'] == 'CPU']
ext_CPU['Supplier'] = np.where(ext_CPU['FV'].str.contains('INTEL', case=False), 'INTEL', 'AMD')
ext_CPU_group = ext_CPU.groupby(['Commodity','Supplier']).size().reset_index(name='Count')
ext_CPU_group['Commodity'] = ext_CPU_group['Supplier'] + ' ' +  ext_CPU_group['Commodity']
ext_CPU_group.drop(columns=['Supplier'], inplace=True)

C:\Users\hsudu\AppData\Local\Temp\ipykernel_19772\3884976303.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ext_CPU['Supplier'] = np.where(ext_CPU['FV'].str.contains('INTEL', case=False), 'INTEL', 'AMD')


### get internal data

In [6]:
# connection
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()
# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview where [Report Date] = '{today}'")
internal = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])
# check if there is empty value in Commodity
if internal.Commodity.isna().any():
    print(internal[internal.Commodity.isna()])
else:
    print('No empty value in Commodity')

No empty value in Commodity


In [7]:
# drop the rows with 'Delete' == 'Y' and group by Commodity
internal = internal[(~(internal['Delete'] == 'Y'))]
int_counts = internal.groupby(['Commodity']).size().reset_index(name='Count')

# CPU need to add supplier
int_CPU = internal[internal['Commodity'] == 'CPU']
int_CPU['Supplier'] = np.where(int_CPU['FV Des'].str.contains('INTEL', case=False), 'INTEL', 'AMD')
int_CPU_group = int_CPU.groupby(['Commodity','Supplier']).size().reset_index(name='Count')
int_CPU_group['Commodity'] = int_CPU_group['Supplier'] + ' ' +  int_CPU_group['Commodity']
int_CPU_group.drop(columns=['Supplier'], inplace=True)

C:\Users\hsudu\AppData\Local\Temp\ipykernel_19772\1167734445.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  int_CPU['Supplier'] = np.where(int_CPU['FV Des'].str.contains('INTEL', case=False), 'INTEL', 'AMD')


In [8]:
shortage_ext.head()

,ODM,Item,Commodity,FV,Platform,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,HP_PN,BuyerName
0,CEI,LCD122,LCD,15.6HD AG SVA SLIMNB,Maldives R6U 2.0/Maldives R6U,9440.0,0.0,0.0,9440.0,0.0,0.0,2024-04-30,2024-05-27,"M02096-3D1,M02096-LD2,M02096-JD3",FerngApple
1,CEI,LCD235,LCD,13.3WQXGA AG UWVA EDPPSR NB sRGB 400N,Conrad 24C1,1385.0,0.0,0.0,1385.0,0.0,0.0,2024-04-30,2024-06-17,"M33458-332,M33458-L31",FerngApple
3,CEI,LCD240,LCD,16WUXGA AG UWVA NTSC FLAT TOP 300N,Motto 24C1/Canopy 24C1,211.0,0.0,0.0,211.0,210.0,0.0,2024-04-30,2024-05-09,"N64563-361,N64563-J61",WangYu-Ting
4,CQIEC,1.0,LCD,15.6FHD UWVA BV Landscape 400N TOP,Tasmania/L91391,20138.0,0.0,0.0,20138.0,0.0,0.0,2024-04-30,2024-07-29,"L91391-3D1,L91391-3D4",FerngApple
6,CQIEC,4.0,LCD,15.6FHD UWVA BV Landscape+CAM 400N TOP,Tasmania,1650.0,0.0,0.0,1650.0,0.0,0.0,2024-04-30,2024-05-02,"L91390-3D1,L91390-3D4",FerngApple


In [9]:
CQODM = ['CQQCI', 'CQIEC', 'CQWIS', 'TWIEC', 'L_WHFXN', 'CQPCQ', 'MSI', 'PCQ', 'PCP', 'WHFXN_L5', 'PSZ', 'FLH', 'TWQCI', 'SZBYD']
WHODM = ['WHFXN']
KSODM = ['CEI']
THODM = ['THQCI','THIEC']
MXODM = ['IMX']

ODM_list = CQODM + WHODM + KSODM + THODM + MXODM

incorrect_ODM = []
for index in internal.index:
    if internal.loc[index,'ODM'] not in ODM_list:
        incorrect_ODM.append(internal.loc[index,:])
    else:
        pass

incorrect_ODM = pd.DataFrame(incorrect_ODM)
try:
    incorrect_ODM = incorrect_ODM[['Buyer Name','Commodity','ODM']]
except:
    pass
print(len(incorrect_ODM))

0


In [10]:
Commodity_list = ['AC ADAPTOR','AudioAMP','AudioCodec','BATTERY','CardReader','CHIPSET','CONNECTOR','CPU','EC','eMMC','Ethernet IC','GPU','GPU-Graphic Card','HDD','LanChip','LCD','Memory','NIC',
 'ODD','POWERCORDS','PSU','Retimer','SIO','SSS','ThunderBT','TPM','TS','USBIC','VRAM','WEBCAM','WLAN','WWAN']

incorrect_commodity = []
for index in internal.index:
    if internal.loc[index,'Commodity'] not in Commodity_list:
        incorrect_commodity.append(internal.loc[index,:])
    else:
        pass

incorrect_commodity = pd.DataFrame(incorrect_commodity)
print(len(incorrect_commodity))

0


### merge and compare internal with external

In [11]:
df_compare = int_counts[int_counts['Commodity'] !='CPU'].merge(ext_count[ext_count['Commodity'] != 'CPU'], on=['Commodity'], how='outer',suffixes=('_int', '_ext'))
df_compare_CPU = int_CPU_group.merge(ext_CPU_group, on=['Commodity'], how='outer',suffixes=('_int', '_ext'))
df_compare = pd.concat([df_compare, df_compare_CPU], axis=0)

In [12]:
df_mismatch = df_compare[((df_compare['Count_int'].isnull()) & (df_compare['Count_ext'].notnull())) | ((df_compare['Count_int'].notnull()) & (df_compare['Count_ext'].isnull()))]
df_mismatch.loc[:,['Count_int','Count_ext']] = df_mismatch.loc[:,['Count_int','Count_ext']].fillna(0)
df_mismatch.set_index('Commodity', inplace=True)
df_mismatch = df_mismatch.astype(int)
df_mismatch.rename(columns={'Count_int':'Count_internal', 'Count_ext':'Count_external'}, inplace=True)
df_mismatch

,Count_internal,Count_external
Commodity,,
AudioCodec,1,0
PSU,8,0
TS,16,0


In [14]:
if (len(df_mismatch) > 0 ) | (len(incorrect_ODM) > 0) | (len(incorrect_commodity) > 0):
    # sent to list
    To_list = 'gps.taiwan.nb.buy-sell@hp.com'
    CC_list = 'spencer.cheng1@hp.com; dustin.hsu1@hp.com'

    # start creating the email
    mail = win32.Dispatch("Outlook.Application").CreateItem(0)
    mail.To = To_list
    mail.CC = CC_list
    mail.Subject = 'Internal/ External Report Misalignment alert' + '<Report date: ' + today + '>'

    # Step 3: Highlight only the Commodity values
    try:
        for _ in df_mismatch.index:
            df_mismatch.loc[_, 'Commodity'] = f'<span style="background-color: yellow;">{_}</span>'
        # df_mismatch['Commodity'] = df_mismatch['Commodity'].apply(lambda name: f'<span style="background-color: yellow;">{name}</span>')
        df_mismatch['Count_internal'] = df_mismatch['Count_internal'].apply(lambda name: f'<font color = Red">{name}</font>' if name == 0 else name)
        df_mismatch['Count_external'] = df_mismatch['Count_external'].apply(lambda name: f'<font color = Red">{name}</font>' if name == 0 else name)
        # df_mismatch.rename(columns={'Count_int':'Count_internal', 'Count_ext':'Count_external'},inplace=True)
        df_mismatch = df_mismatch.reindex(columns=['Commodity','Count_internal','Count_external'])
    except:
        pass

    # df_mismatch = df_mismatch.reindex(columns = ['Commodity','Count_int','Count_ext'])
    # Step 4: Convert the DataFrame to HTML, allowing HTML content within cells
    html_compare_2 = df_mismatch.to_html(escape=False, index=False)
    try:
        incorrect_ODM['ODM'] = incorrect_ODM['ODM'].apply(lambda name: f'<font color = Red">{name}</font>')
    except:
        pass
    try:
        incorrect_commodity['Commodity'] = incorrect_commodity['Commodity'].apply(lambda name: f'<font color = Red">{name}</font>')
    except:
        pass
    # message --> use the html content from the previous step 1&2 to 
    signature = "<br><br>Best Regards,<br>Newcomen Project Team"
    message = f"""Hi team,<br><br>Below information is <b><font color = 'blue'>misaligned/ missing</b></font> in external & internal reports on report date: <b><font color = 'blue'>{today}</b></font>,
        please help to check and make amendment regardingliy.<br>{html_compare_2}<br><br>
        Below data contains <b><font color = 'blue'>incorrect ODM name</b></font>, please delete the data and upload again with the correct ODM name.<br>{incorrect_ODM.to_html(escape=False, index=False)}<br><br>
        Below data contains <b><font color = 'blue'>incorrect Commodity name</b></font>, data is deleted by project team, please upload the data with correct commodity name.<br>{incorrect_commodity.to_html(escape=False, index=False)}<br><br>
        {signature}"""

    # make sure email fully initialized, and only write after <body>
    mail.GetInspector 
    index = mail.HTMLbody.find('>', mail.HTMLbody.find('<body')) 

    # generate email
    mail.HTMLBody = message
    mail.display()

else:
    print('No error found, no email sent')

In [15]:
# connection
conn = pyodbc.connect('Driver={SQL Server Native Client 11.0}; Server=g7w11206g.inc.hpicorp.net; Database=CSI; Trusted_Connection=Yes;')
cursor = conn.cursor()

# download shortge from SQL
cursor.execute(f"SELECT * FROM OPS.GPS_view_ops_critical_shortage_overview where [Report Date] = '{today}'")
shor = pd.DataFrame.from_records(cursor.fetchall(), columns = [i[0] for i in cursor.description])